In the terminal: `cd ..` to be in the `_shared_utils` sub-folder, run `python setup.py install`, then `cd` back into your task sub-folder (e.g., `bus_service_increase`)

In [1]:
import branca
import geopandas as gpd
import pandas as pd

import shared_utils

In [2]:
shared_utils.geography_utils.CA_NAD83Albers

'EPSG:3310'

In [3]:
shared_utils.utils.geoparquet_gcs_export?

Signature: shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, name)
Docstring:
Save geodataframe as parquet locally, 
then move to GCS bucket and delete local file.

gdf: geopandas.GeoDataFrame
GCS_FILE_PATH: str. Ex: gs://calitp-analytics-data/data-analyses/my-folder/
name: str. Filename.
File:      ~/data-analyses/_shared_utils/shared_utils/utils.py
Type:      function


In [ ]:
sample_data = {
    'District' : [1, 2, 3, 1],
    'Population' : [3_000, 2_000, 1_500, 3_000],
    'id' : ['A', 'B', 'C', 'D']
}

df = pd.DataFrame(sample_data)
df

In [ ]:
group_cols = ["District"]
sum_cols = []
mean_cols = ["Population"]
count_cols = []
nunique_cols = ["id"]

shared_utils.geography_utils.aggregate_by_geography(df, group_cols=group_cols,
                                                    sum_cols = sum_cols,
                                                    mean_cols = mean_cols,
                                                    count_cols = count_cols,
                                                    nunique_cols = nunique_cols)

In [ ]:
gdf = gpd.read_parquet((
    "gs://calitp-analytics-data/data-analyses/bus_service_increase/"
    "bus_stop_times_by_tract.parquet"
))
gdf = gdf[gdf.County=="Los Angeles"]

gdf.head()

In [ ]:
popup_dict = {
    "Tract": "Census Tract",
    "ZIP": "Zip Code",
    "Population": "Pop"
}

colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

m = shared_utils.map_utils.make_folium_choropleth_map(gdf,
                                                  plot_col = "Population",
                                                  popup_dict = popup_dict,
                                                  tooltip_dict = popup_dict,
                                                  colorscale = colorscale,
                                                  fig_width = 500, fig_height = 800,
                                                  zoom = 9,
                                                  centroid = shared_utils.map_utils.REGION_CENTROIDS["Los Angeles"][0],
                                                  title="Population in LA"
)

#m